In [45]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import csv
import os.path


In [46]:

findphd_url = 'https://www.findaphd.com/phds/computer-science/non-eu-students/?11M7g40&PG={}'
#keyword_param = 'artificial intelligence'
ind= 1
headers = {
        'Access-Control-Allow-Origin': '*',
        'Access-Control-Allow-Methods': 'GET',
        'Access-Control-Allow-Headers': 'Content-Type',
        'Access-Control-Max-Age': '3600',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1823.51'

    }
phd_df = pd.DataFrame(columns=['Title', 'University', 'Professor', 'Description', 'Deadline'])
url = findphd_url.format(1) # Getting url from page 1 of result
response = requests.get(url, headers)
soup = BeautifulSoup(response.content, 'html.parser')
total_page = soup.find('div', class_='pager').get_text().split('.')[-1] if soup.find('div', class_='pager') is not None else ''
total_page = int(total_page)

for page_num_param in range(total_page):
    url = findphd_url.format(page_num_param+1)
    response = requests.get(url, headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    all_section = soup.find_all('div', class_='resultsRow')
    
    
    for each_sec in all_section:
        if each_sec.find('h3') is not None:
            
            title = each_sec.find('h3').get_text().strip() if each_sec.find('h3') is not None else ''
            university = each_sec.find('a', class_='instLink').get_text().strip() if each_sec.find('a',class_='instLink') is not None else ''
            professor = each_sec.find('a', class_='phd-result__key-info').get_text().strip().split(':')[1] if each_sec.find('a', class_='phd-result__key-info') is not None else ''
            description = each_sec.find('div', class_='descFrag').get_text().split('Read')[0].strip() if each_sec.find('div', class_='descFrag') is not None else ''
            string_with_deadline = each_sec.find('div', class_='phd-icon-area').get_text().strip().split('\n')[0] if each_sec.find('div', class_='phd-icon-area') is not None else ''
            link_to_project = 'https://www.findaphd.com' + each_sec.find('a', class_='h4').get('href') if each_sec.find('a', class_='h4') is not None else ''
            f = open('computer_science_phd.csv', 'a', newline='', encoding="utf-8")
            w = csv.writer(f)
            w.writerow([title, university, professor, description, string_with_deadline, link_to_project])
f.close()
